In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from bayes_opt import BayesianOptimization
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
import color_functions
from color_functions import new_spectra_from_stock
from RL_functions import obtain_data

## Using Actual Data from Excel

In [2]:
Sample_concentrations = pd.read_excel('Sample_concentrations.xlsx')
red_conc = Sample_concentrations[['Red vol frac']].values
green_conc = Sample_concentrations[['Green vol frac']].values
blue_conc = Sample_concentrations[['Blue vol frac']].values
sample_conc = np.hstack((red_conc, green_conc, blue_conc))
y_train = sample_conc[0:-1,:]

#Normalize y_train
for j in range(y_train.shape[0]):
    row_sum = np.sum(y_train[j,:])
    for i in range(y_train.shape[1]):
        y_train[j,i] = y_train[j,i]/row_sum

sample_spectra = pd.read_excel('Sample_spectra.xlsx')
sample_spectra = np.asarray(sample_spectra)
x_train = sample_spectra[:,1:-1]
x_test = sample_spectra[:,-1].reshape(-1,1)

#Normalizae x_train
x_train = MinMaxScaler().fit(x_train).transform(x_train)
x_test = MinMaxScaler().fit(x_test).transform(x_test).T
x_train = x_train.T
x_test = x_test.reshape(1,-1)[0].reshape(-1,1).T

pca = PCA(n_components=0.99)
x_train_new = pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)


## Linear Regression

In [3]:
reg = LinearRegression().fit(x_train, y_train)
prediction_array = reg.predict(x_test)

## Decision Tree Regressor

In [4]:
estimator = DecisionTreeRegressor(max_depth= 79)
clf = BaggingRegressor(base_estimator=estimator, n_estimators=56)
clf = clf.fit(x_train, y_train)
prediction_array = clf.predict(x_test)

## GP Regression

In [5]:
gpr = GaussianProcessRegressor().fit(x_train, y_train)
gpr.predict(x_test, return_std=True)

(array([[0.53449712, 0.00775591, 0.49257213]]), array([0.08186359]))

### Neural Network

In [6]:
regr = MLPRegressor(max_iter=500).fit(x_train, y_train)
prediction_nn = regr.predict(x_test)
red_prediction_nn = prediction_nn[0][0]
green_prediction_nn = prediction_nn[0][1]
blue_prediction_nn = prediction_nn[0][2]
sum_prediction_nn = np.sum(prediction_nn[0])
print(red_prediction_nn/sum_prediction_nn)
print(green_prediction_nn/sum_prediction_nn)
print(blue_prediction_nn/sum_prediction_nn)

0.4797501061127716
0.022024790114073697
0.4982251037731548


### Test the Performance of each Model

In [7]:
nn_y_train = []
dtr_y_train = []
lr_y_train = []
gpr_y_train = [] 

for i in range(x_train.shape[0]):
    prediction_nn = regr.predict(np.array([x_train[i,:]]))
    nn_y_train.append(prediction_nn[0])
    prediction_dtr = clf.predict(np.array([x_train[i,:]]))
    dtr_y_train.append(prediction_dtr[0])
    prediction_lr = reg.predict(np.array([x_train[i,:]]))
    lr_y_train.append(prediction_lr[0])
    prediction_gpr = gpr.predict(np.array([x_train[i,:]]))
    gpr_y_train.append(prediction_gpr[0])

In [8]:
nn_y_train_array = nn_y_train[0]
for i in range(1,len(nn_y_train)):
    nn_y_train_array = np.vstack((nn_y_train_array, nn_y_train[i]))

dtr_y_train_array = dtr_y_train[0]
for i in range(1,len(dtr_y_train)):
    dtr_y_train_array = np.vstack((dtr_y_train_array, dtr_y_train[i]))

lr_y_train_array = lr_y_train[0]
for i in range(1,len(lr_y_train)):
    lr_y_train_array = np.vstack((lr_y_train_array, lr_y_train[i]))

gpr_y_train_array = gpr_y_train[0]
for i in range(1,len(gpr_y_train)):
    gpr_y_train_array = np.vstack((gpr_y_train_array, gpr_y_train[i]))

In [9]:
print('Compare the predicted concentration dataset to the actual')
print('MSE of neural network:', np.sum((nn_y_train_array - y_train)**2))
print('MSE of decision tree regressor:', np.sum((dtr_y_train_array - y_train)**2))
print('MSE of linear regressor:', np.sum((lr_y_train_array - y_train)**2))
print('MSE of gaussian process regressor:', np.sum((gpr_y_train_array - y_train)**2))

Compare the predicted concentration dataset to the actual
MSE of neural network: 0.06554210806768139
MSE of decision tree regressor: 0.3430122434887773
MSE of linear regressor: 0.12320290151219522
MSE of gaussian process regressor: 2.8321745802036165e-16


## Spectra Generator for RBG Dyes 

In [10]:
x,y = obtain_data(50)
Sample_concentrations = pd.read_excel('Sample_concentrations.xlsx')
red_conc = Sample_concentrations[['Red conc']].values
green_conc = Sample_concentrations[['Green conc']].values
blue_conc = Sample_concentrations[['Blue conc']].values
sample_conc = np.hstack((red_conc, green_conc, blue_conc))
sample_spectra = pd.read_excel('Sample_spectra.xlsx')
sample_spectra = np.asarray(sample_spectra)
wavelength = sample_spectra[:,0]
red = sample_spectra[:,1]
green = sample_spectra[:,2]
blue = sample_spectra[:,3]
spectra = []
for i in range(x.shape[0]):
    spectra.append(new_spectra_from_stock(x[i,0],x[i,1],x[i,2] , red, green, blue, wavelength))

In [11]:
spectra0 = spectra[0]
for i in range(1,len(y)):
    spectra0 = np.vstack((spectra0,spectra[i]))

In [12]:
x_train = spectra0 #spectra 
y_train = x #concentrations 

In [13]:
pca = PCA(n_components=0.99)
x_train_new = pca.fit(x_train)
x_train = pca.transform(x_train)

In [14]:
x_test = sample_spectra[:,-1].reshape(-1,1)
x_test = MinMaxScaler().fit(x_test).transform(x_test).T
x_test = pca.transform(x_test)

### GP Regression

In [15]:
gpr = GaussianProcessRegressor().fit(x_train, y_train)
prediction_gpr = gpr.predict(x_test, return_std=True)

In [16]:
red_prediction_gpr = prediction_gpr[0][0][0]
green_prediction_gpr = prediction_gpr[0][0][1]
blue_prediction_gpr = prediction_gpr[0][0][2]
sum_prediction_gpr = np.sum(prediction_gpr[0][0])

In [17]:
print(red_prediction_gpr/sum_prediction_gpr)
print(green_prediction_gpr/sum_prediction_gpr)
print(blue_prediction_gpr/sum_prediction_gpr)

0.37184777932200014
0.33618538261692427
0.29196683806107554


### Linear Regression

In [18]:
reg = LinearRegression().fit(x_train, y_train)
prediction_lr = reg.predict(x_test)
red_prediction_lr = prediction_lr[0][0]
green_prediction_lr = prediction_lr[0][1]
blue_prediction_lr = prediction_lr[0][2]
sum_prediction_lr = np.sum(prediction_lr[0])

In [19]:
print(red_prediction_lr/sum_prediction_lr)
print(green_prediction_lr/sum_prediction_lr)
print(blue_prediction_lr/sum_prediction_lr)

0.5180415502656843
-0.04837885156910791
0.5303373013034236


### Decision Tree Regression

In [20]:
estimator = DecisionTreeRegressor(max_depth=27)
clf = BaggingRegressor(base_estimator=estimator, n_estimators=43)
clf = clf.fit(x_train, y_train)
prediction_dtr = clf.predict(x_test)

In [21]:
red_prediction_dtr = prediction_dtr[0][0]
green_prediction_dtr = prediction_dtr[0][1]
blue_prediction_dtr = prediction_dtr[0][2]
sum_prediction_dtr = np.sum(prediction_dtr[0])
print(red_prediction_dtr/sum_prediction_dtr)
print(green_prediction_dtr/sum_prediction_dtr)
print(blue_prediction_dtr/sum_prediction_dtr)

0.46082033537068096
0.13239639605315798
0.4067832685761611


### Neural Network 

In [22]:
regr = MLPRegressor(max_iter=500).fit(x_train, y_train)
prediction_nn = regr.predict(x_test)

In [23]:
red_prediction_nn = prediction_nn[0][0]
green_prediction_nn = prediction_nn[0][1]
blue_prediction_nn = prediction_nn[0][2]
sum_prediction_nn = np.sum(prediction_nn[0])
print(red_prediction_nn/sum_prediction_nn)
print(green_prediction_nn/sum_prediction_nn)
print(blue_prediction_nn/sum_prediction_nn)

0.4818201797272399
0.14177108881802467
0.3764087314547355


### MSE of each model 

In [24]:
nn_y_train = []
dtr_y_train = []
lr_y_train = []
gpr_y_train = [] 

for i in range(x_train.shape[0]):
    prediction_nn = regr.predict(np.array([x_train[i,:]]))
    nn_y_train.append(prediction_nn[0])
    prediction_dtr = clf.predict(np.array([x_train[i,:]]))
    dtr_y_train.append(prediction_dtr[0])
    prediction_lr = reg.predict(np.array([x_train[i,:]]))
    lr_y_train.append(prediction_lr[0])
    prediction_gpr = gpr.predict(np.array([x_train[i,:]]))
    gpr_y_train.append(prediction_gpr[0])
  

In [25]:
nn_y_train_array = nn_y_train[0]
for i in range(1,len(nn_y_train)):
    nn_y_train_array = np.vstack((nn_y_train_array, nn_y_train[i]))

dtr_y_train_array = dtr_y_train[0]
for i in range(1,len(dtr_y_train)):
    dtr_y_train_array = np.vstack((dtr_y_train_array, dtr_y_train[i]))

lr_y_train_array = lr_y_train[0]
for i in range(1,len(lr_y_train)):
    lr_y_train_array = np.vstack((lr_y_train_array, lr_y_train[i]))

gpr_y_train_array = gpr_y_train[0]
for i in range(1,len(gpr_y_train)):
    gpr_y_train_array = np.vstack((gpr_y_train_array, gpr_y_train[i]))

In [26]:
print('Compare the predicted concentration dataset to the actual')
print('MSE of neural network:', np.sum((nn_y_train_array - y_train)**2))
print('MSE of decision tree regressor:', np.sum((dtr_y_train_array - y_train)**2))
print('MSE of linear regressor:', np.sum((lr_y_train_array - y_train)**2))
print('MSE of gaussian process regressor:', np.sum((gpr_y_train_array - y_train)**2))

Compare the predicted concentration dataset to the actual
MSE of neural network: 1.4429936778609231
MSE of decision tree regressor: 0.2500033475177036
MSE of linear regressor: 4.6484245137730325e-30
MSE of gaussian process regressor: 2.0850025844287215e-19


## Test Model with actual Data

In [48]:
Sample_concentrations = pd.read_excel('Sample_concentrations.xlsx')
red_conc = Sample_concentrations[['Red vol frac']].values
green_conc = Sample_concentrations[['Green vol frac']].values
blue_conc = Sample_concentrations[['Blue vol frac']].values
sample_conc = np.hstack((red_conc, green_conc, blue_conc))
y_train = sample_conc[0:-1,:]

#Normalize y_train
for j in range(y_train.shape[0]):
    row_sum = np.sum(y_train[j,:])
    for i in range(y_train.shape[1]):
        y_train[j,i] = y_train[j,i]/row_sum

sample_spectra = pd.read_excel('Sample_spectra.xlsx')
sample_spectra = np.asarray(sample_spectra)
x_train = sample_spectra[:,1:-1]
x_test = sample_spectra[:,-1].reshape(-1,1)

#Normalizae x_train
x_train = MinMaxScaler().fit(x_train).transform(x_train)
x_test = MinMaxScaler().fit(x_test).transform(x_test).T
x_train = x_train.T
x_test = x_test.reshape(1,-1)[0].reshape(-1,1).T

pca = PCA(n_components=0.99)
x_train_new = pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)


In [49]:
prediction_gpr = gpr.predict(x_train)

In [50]:
prediction_lr = reg.predict(x_train)

In [51]:
prediction_dtr = clf.predict(x_train)

In [52]:
prediction_nn = regr.predict(x_train)

In [57]:
print('MSE of GPR:', np.sum((y_train - prediction_gpr)**2))
print('MSE of LR:', np.sum((y_train - prediction_lr)**2))
print('MSE of DTR:', np.sum((y_train - prediction_dtr)**2))
print('MSE of NN:', np.sum((y_train - prediction_nn)**2))

MSE of GPR: 6.782500593252865
MSE of LR: 6.840778948942412
MSE of DTR: 6.857393791187935
MSE of NN: 5.804235096937878
